In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
land_specs_df = pd.read_csv('./토지특성정보/AL_11_D195_20200925.csv', encoding='euc-kr')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def landnum_modifier(x):
    # x is a string
    
    splitted = x.split('-')
    if len(splitted) == 1:
        return x+'-0'
    elif len(splitted) == 2:
        return x
    else:
        raise ValueError('myerror')

In [5]:
def prep_production_ready_land_specs(land_specs_df):
    #
    
    land_specs_df = land_specs_df.copy()
    
    land_specs_df.drop(columns=[col for col in land_specs_df.columns if '코드' in col], inplace=True)
    land_specs_df['지번주소'] = land_specs_df['법정동명'] + ' ' + land_specs_df['지번'].apply(landnum_modifier)
    land_specs_df.drop(columns=['고유번호', '법정동명', '지번', '대장구분명', '토지일련번호', '기준월', '데이터기준일자'], inplace=True)
    land_specs_df = land_specs_df.sort_values(['지번주소', '기준년도']).reset_index(drop=True)
    land_specs_df.drop_duplicates(subset=['지번주소'], keep='last', inplace=True)
    land_specs_df.drop(columns=['기준년도', '공시지가'], inplace=True)
    
    return land_specs_df

In [6]:
land_specs_production_ver_1 = prep_production_ready_land_specs(land_specs_df)

In [7]:
print(land_specs_production_ver_1.shape)
land_specs_production_ver_1.head()

(983625, 9)


,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,지번주소
7,전,876.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,서울특별시 강남구 개포동 100-0
15,전,641.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 101-0
23,전,509.0,자연녹지지역,지정되지않음,전,평지,부정형,맹지,서울특별시 강남구 개포동 102-0
31,전,48.7,자연녹지지역,개발제한구역,전,평지,부정형,맹지,서울특별시 강남구 개포동 103-0
39,전,2995.0,개발제한구역,자연녹지지역,전,완경사,부정형,맹지,서울특별시 강남구 개포동 104-1


In [8]:
land_specs_production_ver_1.to_csv('./토지특성정보/land_specs_production_ver_1.csv', index=False)